In [1]:
!pip install -q langchain langchain-nvidia-ai-endpoints langchain-groq langchain-community langchain-core chromadb langchain-google-genai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.7 MB/s eta 0:

In [2]:
import os

from google.colab import userdata
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [21]:
# require libraries

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [22]:
# Hypothetical embeddings
hyde_template = """Please write a scientific passage paper to answer the question.

Question: {question}

Passage:
"""

hyde_prompt = ChatPromptTemplate.from_template(hyde_template)

# llm = ChatGroq(model_name='llama-3.1-70b-versatile')
llm = ChatNVIDIA(model='meta/llama-3.1-8b-instruct')

chain = hyde_prompt | llm

In [23]:
chain

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Please write a scientific passage paper to answer the question.\n\nQuestion: {question}\n\nPassage:\n'))])
| ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='meta/llama-3.1-8b-instruct')

In [24]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

hyde_embeddings = HypotheticalDocumentEmbedder.from_llm(
    chain,embeddings,prompt_key='web_search'
)

In [25]:
hyde_embeddings

HypotheticalDocumentEmbedder(base_embeddings=GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x77fcf8e0a650>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None), llm_chain=PromptTemplate(input_variables=['QUESTION'], template='Please write a passage to answer the question \nQuestion: {QUESTION}\nPassage:')
| ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Please write a scientific passage paper to answer the question.\n\nQuestion: {question}\n\nPassage:\n'))])
| ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='meta/llama-3.1-8b-instruct')
| StrOutputParser())

In [26]:
# load data
loader = PyPDFLoader('/content/RAG-for NLP.pdf')
data = loader.load()

In [27]:
# split the data
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = splitter.split_documents(data)

In [28]:
# Vector store
vectorstore = Chroma.from_documents(chunks,hyde_embeddings)

In [29]:
# retriever
retriever = vectorstore.as_retriever()

In [30]:
# RAG
template = """Answer the following based only on the provided context.
If the answer is not in the context, say "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# llm = ChatGroq(model_name='llama-3.1-70b-versatile')
llm = ChatNVIDIA(model='meta/llama-3.1-8b-instruct')

retrieval_chain = ({'context':retriever, 'question':RunnablePassthrough()}
                   |prompt
                   |llm
                   |StrOutputParser())

In [31]:
retrieval_chain.invoke('Explain the Jeopardy Question Generation')

'According to the provided context, Jeopardy Question Generation is an unusual format that consists of trying to guess an entity from a fact about that entity. For example, "The World Cup" is the answer to the question "In 1986 Mexico scored as the first country to host this international sports competition twice." It is described as a challenging knowledge-intensive generation task because Jeopardy questions are precise, factual statements.\n\nThe context also mentions that the task is used to evaluate the performance of the RAG model, which is compared to the state-of-the-art generation model BART. The evaluation is done using the SQuAD-tuned Q-BLEU-1 metric, as well as human evaluations to assess generation factuality and diversity.'

In [32]:
retrieval_chain.invoke('what is Open-domain Question Answering?')

'Open-domain Question Answering is a type of question-answering task where the goal is to find a valid answer to a question given the knowledge contained in a large document collection, without any domain constraints.'

In [33]:
retrieval_chain.invoke('explain the Fact Verification in detail')

'Based on the provided context, Fact Verification is discussed in the "3.4 Fact Veriﬁcation" section. Here\'s an explanation in detail:\n\n**Task: FEVER**\n\nThe task referred to is called FEVER (Fact Extraction and VERification). It is a natural language understanding task that requires classifying whether a given natural language claim is supported or refuted by Wikipedia, or whether there is not enough information to decide.\n\n**Task Requirements**\n\nThe task has the following requirements:\n\n1. **Retrieving Evidence**: The model is required to retrieve evidence from Wikipedia that is related to the claim.\n2. **Reasoning over Evidence**: After retrieving the evidence, the model needs to reason over this evidence to classify the claim as supported, refuted, or insufficient information.\n\n**Task Objective**\n\nThe objective of FEVER is to evaluate the model\'s ability to perform fact verification, which involves checking the truthfulness of a given statement or claim.\n\nThat\'s 